## Introduktion / Problemställning

Projektet genomfördes inom kursen _Djup maskininlärning_ som en gruppuppgift med fri problemformulering. Ett av flera föreslagna fokusområden var **Spelande AI** med reinforcement learning, och detta valdes för projektet. Vid sökandet efter en lämplig spelmiljö identifierades [MiniGrid](https://minigrid.farama.org/), som valdes eftersom den upplevdes som enkel att förstå och arbeta med.

MiniGrid är ett bibliotek med enkla gridbaserade miljöer för reinforcement learning, där en agent med hjälp av sina tillgängliga handlingar och observationer ska navigera mot ett mål i tvådimensionella rutnät. Biblioteket innehåller flera färdiga miljöer med olika hinder och svårighetsgrader, och det är även möjligt att själv konfigurera miljöer och banor för agenten att lösa.

I detta projekt formulerades följande problemställning:

**Kan en modell som tränats på slumpmässigt genererade MiniGrid-banor klara av förbyggda, tidigare osedda banor?**

---

## Data-analys / Miljöförståelse

För att förstå MiniGrid-miljön analyserades agentens möjliga handlingar, observationer, objekt och regler.

Banorna som agenten ska lösa kan innehåller hinder och objekt såsom väggar, dörrar, lava och mål. Agenten kan utföra sju diskreta handlingar: svänga vänster, svänga höger, gå framåt, plocka upp objekt, släppa objekt, aktivera objekt samt avsluta episoden. Beroende på miljö kan det dock ställas in vilka av dessa handlingar som faktiskt är tillgängliga.

Agentens tre tillgängliga observationer är **image**, **direction** och **mission**. **Image** visar hur rutnätet framför agenten ser ut, **direction** anger agentens nuvarande riktning i banan, och **mission** är en sträng som kan innehålla information om målet eller uppgiften som agenten ska lösa.

Som nämnt tidigare går det att konfigurera egna banor utöver de färdiga. Det är även möjligt att ändra vissa inställningar för agenten, till exempel synfältets storlek och om väggar blockerar sikten. MiniGrid innehåller dessutom färdiga "wrappers" som kan påverka agentens beteende, exempelvis filtrering av observationer eller belöning för utforskning.

Analysen gav en tydlig bild av miljön och agentens förutsättningar, vilket lade grunden för banornas utformning, modellutvecklingen och själva inlärningen.

---

## Metod

Detta avsnitt beskriver hur experimenten genomfördes, från miljöer och banor till modeller och träning.

### Miljökonfiguration

För träning skapades en klass som utgick från basklassen `MiniGridEnv` och genererade banor med mål, väggar och lava på slumpmässiga positioner vid varje episod. Syftet med denna variation var att agenten skulle lära sig att navigera i olika miljöer och potentiellt klara av banor som den inte stött på under träning.

Nedan visas exempel på slumpmässigt genererade träningsbanor:

<img src="./images/procedural_levels_example.png" width="860px">

För testning skapades en separat klass, även den baserad på `MiniGridEnv`, som läste in förbyggda banor. Dessa banor användes för att utvärdera modellens förmåga att lösa banor den inte tidigare tränats på. Olika typer av banor byggdes för att variera svårighet och utmaning i navigering.

Nedan visas de förbyggda testbanorna:

<img src="./images/fixed_test_levels.png" width="860px">

I båda dessa konfigurerade miljöer begränsades agentens handlingar till att endast svänga vänster, svänga höger och gå framåt. Detta beslut togs eftersom varken tränings- eller testbanor innehöll dörrar eller nycklar, och syftet var att undvika att agenten använde handlingar som inte hade någon effekt.

### Modell

Två olika algoritmer användes vid inlärning i detta projekt.

Till en början testades **PPO** från *StableBaseline3*, eftersom denna presenterades som exempel i MiniGrid-dokumentationen: [Training Minigrid Environments](https://minigrid.farama.org/content/training/). Vid användning av **PPO** observerades att agenten ofta kunde utforska samma vägar flera gånger, vilket ledde till en undersökning av **RecurrentPPO**.

**RecurrentPPO** från *sb3-contrib* är en vidareutveckling av **PPO** som stöder rekurrenta policies genom användning av **Long Short-Term Memory (LSTM)**-nätverk. En rekurrent policy kan ta hänsyn till tidigare steg, vilket gör att agenten kan komma ihåg tidigare observationer och fatta bättre beslut.

I alla inlärningsexperiment användes wrappern _ImgObsWrapper_, som gör att agenten enbart använder **image** som observation. Vid slutgiltig inlärning med **RecurrentPPO** användes även _OneHotPartialObsWrapper_, som omvandlar agentens synfält till en one-hot-kodad representation.

För **PPO** användes `"CnnPolicy"`, medan **RecurrentPPO** använde `"CnnLstmPolicy"` för att möjliggöra rekurrenta policies med LSTM-nätverk. För båda algoritmerna specificerades inställningar för policyn, där `features_extractor_class` och `features_dim` följde MiniGrid-dokumentationen. Till **RecurrentPPO** sattes även `normalize_images=False` för att behålla råa pixelvärden som input.

### Belöningsfunktion

I projektet testades endast en belöningsfunktion, vilken ärvdes direkt från `MiniGridEnv`. Denna ger endast belöning när agenten når målet, och den slutgiltiga belöningen beror på hur många steg agenten tar under en episod. Belöningsfunktionen definieras som:

$$
reward = 1 - \frac{steps\_taken}{max\_steps}
$$

där `steps_taken` motsvarar antalet handlingar som agenten utför i miljön och `max_steps` anger det maximala tillåtna antalet steg per episod.

Med denna belöningsfunktion uppmuntras agenten att alltid försöka ta den snabbaste vägen till målet.

### Träning

Vid träningen användes klassen för att generera slumpmässiga banor tillsammans med **RecurrentPPO**, och modellen tränades under 3 miljoner timesteps. Nedan visas diagram som illustrerar hur belöningen förändrades över tid i träningen, där ett scatter-diagram visar alla individuella belöningsvärden och ett linjediagram visar medelbelöningen:

<img src="./images/RPPO_training_plot.png" width="640px">

Som framgår av linjediagrammet ligger medelbelöningen (_Mean Reward_) på ungefär samma nivå redan från cirka 300 000 timesteps, vilket tyder på att modellen inte visar någon tydlig förbättring efter detta skede.

Som ett jämförande experiment genomfördes en alternativ träning med **PPO** på en specifik testbana. **Level 4** valdes eftersom denna bedömdes vara utmanande för agenten och innehöll både lava och svängar. Denna modell tränades under 1 miljon timesteps, och belöningen över tid i träningen visas i följande diagram:

<img src="./images/PPO_training_plot.png" width="640px">

Dessa diagram visar att agenten till en början inte klarade banan, men att den över tid lärde sig att genomföra majoriteten av försöken framgångsrikt.

---

## Resultat

Syftet med detta experiment var att undersöka om en modell som tränats på slumpmässigt genererade MiniGrid-banor kan klara av förbyggda, tidigare osedda banor.

<img src="./images/RPPO_success_rate_result.png" width="300px">

Som framgår av stapeldiagrammet ovan, som visar resultaten för modellen tränad på slumpmässiga banor med **RecurrentPPO**, klarade agenten endast **Level 1** och **Level 3**. Vid visuell granskning av agentens beteende på **Level 3** observerades att den tog lång tid på sig att nå målet och ofta gick fram och tillbaka på samma rutor innan den lyckades. När modellen istället testades i nya slumpmässigt genererade banor av samma typ som den tränades på, visade agenten däremot bättre förmåga att hitta målet, även om den inte alltid lyckades.

Den alternativa modellen, tränad med **PPO** endast på **Level 4**, visade inte bättre resultat. Den klarade enbart den bana den tränades på, vilket visar att den inte lärt sig att navigera på nya banor. Även om målet är synligt för agenten vid startpositionen i **Level 1**, har den inte lärt sig att den ska gå dit.

Utifrån problemställningen, där målet var att få en modell att klara tidigare osedda banor, visar resultatet att träning på slumpmässiga banor inte automatiskt leder till framgång på sådanna banor.

---

## Diskussion

I detta projekt lyckades modellen inte klara alla de förbyggda, tidigare osedda banorna. Modellen som tränades med **RecurrentPPO** presterade begränsat, vilket överensstämde med belöningsdiagrammen under inlärning. Dessa diagram visade ingen tydlig förbättring över tid, och indikerade att modellen även efter lång träning hade många misslyckade försök samt relativt låga belöningsvärden.

En stor anledning till att belöningsdiagrammen ser ut som de gör är att de slumpmässiga banorna fick mycket varierande svårhetsgrad, även om antalet utplacerade objekt var detsamma. Det fanns ingen begränsning för hur nära agenten målet kunde placeras. Det hade varit intressant att undersöka hur olika sätt att generera slumpmässiga banor påverkat både träning och resultat. En förbättrad generering av slumpmässiga banor, där mycket lätta banor undviks, skulle troligtvis ha lett till större variation i belöningsdiagrammet och gjort det möjligt för modellen att bättre hantera svårare banor.

Experimentet med **PPO**-modellen som tränades enbart på **Level 4** visade ett tydligt exempel på överanpassning. Modellen klarade endast den bana den tränats på och genomförde den på snabbaste möjliga sätt utan att behöva utforska. Detta visar att den enbart hade lärt sig mönstret för **Level 4**, snarare än att aktivt söka efter målet. Att modellen dessutom inte kunde klara **Level 1**, som ansågs vara den lättaste banan, understryker detta tydligt. Experimentet med träning på endast en bana visar därmed att variation i träningsbanorna är nödvändig.

Utöver förbättrad generering av träningsbanor fanns flera utvecklingsmöjligheter som inte hanns med på grund av tidsbegränsningar. Exempelvis skulle det ha varit värdefullt att bygga mer robusta pipelines för inlärning, avsätta mer tid till experiment med olika modeller, testa alternativa belöningsfunktioner samt förbättra kodbasens struktur.

Vid vidare utveckling av träningsmiljöer, modeller och belöningsfunktioner kan betydligt bättre resultat förväntas uppnås.

---